# Reconstruction (without truncations)
_Author: Lucas Z. Brito_

This notebook reconstructs the Haldane Shastry Hamiltonian. 

### Reconstruction and model parameters

In [113]:
n = 14 # Chain size 
atol = 1e-2 # Absolute error for sanity checks 
output_dir = "../data/hs_data/"

true

In [114]:
using JLD
using DataFrames
using CSV

### Reconstruction

In [115]:
include("./src/hs_tools.jl")

# Make wavefunction and correlation matrix and save to a directory to avoid 
# running calculation again. 
# make_wf_cm(n, output_dir) # Can take roughly a minute for n > 10. 

ft_mat

In [116]:
cm_wf = load(output_dir * "cm_wf_n=$(n).jld")
k = cm_wf["wf"]
cm = cm_wf["cm"]

# Haldane shastry coefficients
hs = normalize(hs_coeffs(n))
hs

7-element Vector{Float64}:
 0.9530925643018818
 0.2506860002798336
 0.1213998418206907
 0.07720590544912054
 0.058137621516648516
 0.04965143681745279
 0.047192917818300865

In [117]:
# Make correlation matrix
# cm = make_corr_mat(n, k)

printstyled("Correlation matrix: ", bold=true); flush(stdout)
display(cm)

# Diagonalize correlation matrix
eig = eigen(cm)

printstyled("Eigenvectors:", bold=true); flush(stdout)
display(eig.vectors)
printstyled("Eigenvalues:", bold=true); flush(stdout)
display(eig.values)

Correlation matrix: 

7×7 Matrix{Float64}:
  0.000154086  -0.00113149   0.00155612  …  -0.000890853   0.000876484
 -0.00113149    0.00859663  -0.0124836       0.00624265   -0.00600894
  0.00155612   -0.0124836    0.0202904      -0.00988669    0.00941036
 -0.00111394    0.0087682   -0.015994        0.0150265    -0.0137606
  0.000987839  -0.00698792   0.0118126      -0.0229371     0.02038
 -0.000890853   0.00624265  -0.00988669  …   0.0286341    -0.0323772
  0.000876484  -0.00600894   0.00941036     -0.0323772     0.0429597

Eigenvectors:

7×7 Matrix{Float64}:
 -0.189141   0.962472    0.163905   …   0.0419732  -0.0414549   0.0232851
 -0.3878     0.0984516  -0.576335      -0.349174    0.346919   -0.171448
 -0.424366  -0.0605815  -0.444381       0.448966   -0.581393    0.280896
 -0.436865  -0.114944   -0.0736056      0.225626    0.408426   -0.356114
 -0.442258  -0.138399    0.241724      -0.664731   -0.138311    0.445181
 -0.444659  -0.148838    0.544438   …   0.0887131  -0.26141    -0.502032
 -0.222677  -0.0759312   0.28851        0.417254    0.534446    0.560465

Eigenvalues:

7-element Vector{Float64}:
 -1.878734222777117e-16
  1.0916773795330031e-15
  0.00045451675152171516
  0.0023149155419541372
  0.011409607476143346
  0.02880047658991191
  0.103483444260802

In [118]:
# Check that conserved quantity and Hamiltonian are in nullspace of CM
nullsp = eig.vectors[:, 1:2]

h_proj, h_is_in_nsp = in_subspace(nullsp, hs, atol)
# in_subspace([eig.vectors[:, 1]], hs, atol)

conserved_quant = normalize(ones(trunc(Int, n / 2)))
stot_proj, stot_is_in_nsp = in_subspace(nullsp, conserved_quant, atol)

# Project HS coefficient onto nullspace; this will be our "reconstructed 
# Hamiltonian."
coeffs_recon = nullsp * inv(nullsp' * nullsp) * nullsp' * hs

if h_is_in_nsp
	printstyled("Hamiltonian is in nullspace ✓", bold=true, color=:green)
else 
	printstyled("Hamiltonian is not in nullspace ×", bold=true, color=:red)
end 

println()

if stot_is_in_nsp
	printstyled("S_tot is in nullspace ✓", bold=true, color=:green)
else 
	printstyled("S_tot is not in nullspace ×", bold=true, color=:red)
end 

println()
println("(atol = $atol)")

Hamiltonian is in nullspace ✓
S_tot is not in nullspace ×
(atol = 0.01)


In [119]:
printstyled("Projections onto nullspace:\n", bold=true)
print("H:\t")
print(h_proj)
println()
print("S_tot:\t")
print(stot_proj)
println()

Projections onto nullspace:
H:	0.9994740259206449
S_tot:	0.9662625197679772


In [120]:
# Save data 
begin
	df = DataFrame(r=1:(trunc(Int, n / 2)), 
				   n=n,
				   coeffs_anal=hs, 
				   coeffs_recon=real.(coeffs_recon))
	fn = CSV.write(output_dir * "hs_recon_n=$n.csv", df)
	printstyled("Written reconstruction data to $fn.", bold=true, color=:green)
end

Written reconstruction data to ../data/hs_data/recon_n=14.csv.